In [25]:
import os
import trimesh

import numpy as np

from mesh_tools import *

In [27]:
# Fixed seed for deterministic results
np.random.seed(0)

# Meshes, loading and processing

Before starting with the implementation of the EMU algorithm for muscle simulation, we must start off by loading our tetrahedralized mesh.
Let us do so.

In [11]:
# TODO: Change to load tetrahedra instead of triangle!

def load_obj_mesh(mesh_path):
    vertex_data = []
    face_data = []
    for line in open(mesh_path, "r"):
        if line.startswith('#'):
            continue
        values = line.split()
        if not values:
            continue
        if values[0] == 'v':
            v = list(map(float, values[1:4]))
            vertex_data.append(v)
        elif values[0] == 'f':
            f = list(map(lambda x: int(x.split('/')[0]),  values[1:4]))
            face_data.append(f)
    vertices = np.array(vertex_data)
    faces = np.array(face_data)
    return vertices, faces 

In [12]:
mesh_path = f"{os.getcwd()}/../data/arm_tet.obj"
vertices, faces = load_obj_mesh(mesh_path)
mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

Once we have loaded our already tetrahedralized mesh, it is important to create a numpy `ndarray` variable of dimensions `(n, 4, 3)`.

The dimensions of this array are as follows:
- `n`: Number of tetrahedra in our mesh
- `4`: Tetrahedra vertices
- `3`: Coordinate of each vertex

Let us name this variable `Q`, to refer to the mesh at its rest position.

# Physics implementation

Now that we have loaded our mesh `Q`, let us begin work on the physics implementation of the EMU algorithm. We will start off by defining the functions that are relevant to the algorithm.

We will define:
- `F`: Deformation gradient
- `G`: Inverse of rest position mesh (`G`)
- `q`: Activated mesh

The equation we seek to minimize is the following equation, $Eq.1$:

$$
\int_{\Omega}{\Psi_{iso}(F(q)) + \Psi_{fiber}(F(q), u, a(t)) - W(q) dQ}
$$

In [18]:
# Number of tetrahedra
# TODO: Review the shape of the tetrahedra
n = 10000
n_vertex = 3 # 4 for tetrahedra, 3 for triangle
n_coords = 3 # x, y, z

tet_shape = (n, n_vertex, n_coords)
dir_shape = (n, n_coords, n_vertex)

F = np.random.random(tet_shape)
Q = np.random.random(tet_shape)
G = np.linalg.inv(Q)

u = np.random.random(dir_shape)

## Linear activation function

We define a linear activation function $a(t), \forall t>0$.

As we will have to derive $\Psi_{fiber}$ later on, we will also define the derivative of our activation function.

In [19]:
def a(t : int) -> float:
    return (1/10) * t # 1/10 for "slow" activation

In [20]:
def d_a() -> float:
    return (1/10)

## $\Psi_{fiber}$ functions

The $\Psi_{fiber}$ function, and its derivatives will be necessary to determine the behaviour of our muscle mesh.

We define the following functions:
- `psi_fiber`: Corresponds to $\Psi_{fiber}$, such that:
$$
\Psi_{fiber}(F, u, a(t)) = a(t)u_{i}^{T}F_{i}^{T}F_{i}u_{i}
$$
- `d_psi_fiber`: Corresponds to $\frac{d\Psi_{fiber}}{dF}$
- `psi_fiber_hessian`: Corresponds to $\frac{d^2\Psi_{fiber}}{dF^2}$

In [21]:
# Equation (8)
def psi_fiber(F : np.ndarray, i : int, t : int) -> np.ndarray:
    # TODO: Still incomplete! Define u unit-length directional vector
    return a(t) * F[i].T @ F[i]

def d_psi_fiber(F : np.ndarray, i : int, t : int) -> np.ndarray:
    # TODO: IMPLEMENT
    pass

def psi_fiber_hessian(F : np.ndarray, i : int, t : int) -> np.ndarray:
    # TODO: IMPLEMENT
    pass

## $\Psi_{iso}$ functions

The $\Psi_{iso}$ function, and its derivatives will be necessary to determine the material of our bone mesh.

We define the following functions:
- `psi_iso`: Corresponds to $\Psi_{iso}$, such that 
$$
\Psi_{iso}(F) = C(tr(F^{T}F) - 3)
$$
- `d_psi_iso`: Corresponds to $\frac{d\Psi_{iso}}{dF}$
- `psi_iso_hessian`: Corresponds to $\frac{d^2\Psi_{iso}}{dF^2}$

In [22]:
def psi_iso(F : np.ndarray, i : int) -> np.ndarray:
    C = np.pi # Material constant
    I = np.trace(F[i].T @ F[i])
    return C * (I - 3)

def d_psi_iso(F : np.ndarray, i : int) -> np.ndarray:
    # TODO: IMPLEMENT
    pass

def psi_iso_hessian(F : np.ndarray, i : int) -> np.ndarray:
    # TODO: IMPLEMENT
    pass

## $E_c$ functions

We must now define the minimized energy function $E_c$ which finds the optimal deformed vertex positions for `q`.

We define the following function:
- `E_c`: Corresponds to $E_c$, such that:
$$
E_c(F) = (G^{T}G)^{-1}G^{T}F
$$
- `d_E_c`: Corresponds to $\frac{dE_c}{dF}$, such that:
$$
\frac{dE_{c}(F)}{dF} = -G(G^{T}G)^{-1}G^{T}F+F
$$

In [23]:
# Equation (4)
def E_c(F : np.ndarray, G : np.ndarray, i : int) -> np.ndarray:
    return np.linalg.inv(G[i].T @ G[i]) @ G[i].T @ F[i]

# Equation (10)
def d_E_c(F : np.ndarray, G : np.ndarray, i : int) -> np.ndarray:
    return -G[i] @ np.linalg.inv(G[i].T @ G[i]) @ G[i].T @ F[i] + F[i]

Now we define the discretized energy minimization from $Eq. 1$.

We define the following functions:
- `E`: Corresponds to $E(F)$, such that: 
$$
E(F) = \Psi_{iso}(F) + \Psi_{fiber}(F,u,a(t)) + \alpha E_c(F) - W(q(F))
$$
- `d_E`: Corresponds to $\frac{dE}{dF}$, such that:
$$
\frac{dE}{dF} = \frac{d\Psi_{iso}}{dF} + \frac{\partial \Psi_{fiber}}{\partial F} + \alpha \frac{d E_c}{d F}
$$
- `E_hessian`: Corresponds to $\frac{d^2E}{dF^2}$, such that:
$$
\frac{d^2E}{dF^2} = \frac{d^2\Psi_{iso}}{dF^2} + \frac{\partial ^2 \Psi_{fiber}}{\partial F} + \alpha I - \alpha (\Phi G^{T})^T \Lambda ^ {-1} (\Phi G^{T})
$$
- `E_hessian_inv`: Corresponds to $(\frac{d^2E}{dF^2})^{-1}$

In [24]:
# Equation (5)
def E(F : np.ndarray, G : np.ndarray, i : int, t : int, α : int) -> np.ndarray:
    return psi_iso(F, i) + psi_fiber(F, i, t) + α * E_c(F, G, i)

# Equation (9)
def d_E(F : np.ndarray, G : np.ndarray, i : int, t : int, α : int) -> np.ndarray:
    return d_psi_iso(F, i) + d_psi_fiber(F, i, t) + α * d_E_c(F, G, i)

# Equation (11, 12, 13, 14, 15)
def E_hessian(F : np.ndarray, G : np.ndarray, i : int, t : int, α : int) -> np.ndarray:
    #(G.T @ G)^-1 = Φ.T Λ^-1 Φ
    Λ, Φ  = np.linalg.eig(G)
    
    I = np.identity(tet_shape)
    H = psi_iso_hessian(F, i) + psi_fiber_hessian(F, i, t) + α * I

    B = Φ @ G[i].T

    # Hessian matrix of E
    hessian = H - α * B.T @ np.linalg.inv(Λ) @ B

    return hessian

# Equation (17)
def E_hessian_inv(F : np.ndarray, G : np.ndarray, i : int, t : int, α : int) -> np.ndarray:
    #(G.T @ G)^-1 = Φ.T Λ^-1 Φ
    Λ, Φ  = np.linalg.eig(G)
    
    I = np.identity(tet_shape)
    H = psi_iso_hessian(F, i) + psi_fiber_hessian(F, i, t) + α * I
    H_inv = np.linalg.inv(H)

    B = Φ @ G[i].T

    # Inverse of hessian expression
    hessian_inv = H_inv + α * H_inv @ B.T @ np.linalg.inv(Λ - α*B @ H_inv @ B.T) @ B @ H_inv

    return hessian_inv

## EMU Implementation

Once we've defined all of the functions necessary for the algorithm to function, we can now define the EMU algorithm.

In [ ]:
# EMU Algorithm implementation

# TODO: How to separate bone mesh and muscle mesh?
#       How will the algo be able to tell the difference between
#       the two when just looking at the tetrahedra's vertices' coordinates?
#       Refer to section 3.6 (Modeling). 
#       c.f: Young's Modulus, labeling (muscle, tendon, bone), Poisson's Ratio

def EMU(F : np.ndarray, G : np.ndarray):
    # Line search step size
    sigma = 10

    # Line search f tolerance
    c = 1e-4

    # Line search decrement
    p = 0.5
    
    # Hyperparameter
    α = 10

    # Number of tetrahedra
    n = G.shape[0]

    # Array to store our new mesh in
    q = np.copy(G)
    
    # TODO: Change this condition.
    for t in range (100):
        F_temp = np.copy(F.shape)
        
        # Iterate on each tetrahedra
        for i in range(n):
            e_i = E(F, G, i, t, α)
            g = d_E(F, G, i, t, α)
            H_inv = E_hessian_inv(F, G, i, t, α)
            d = H_inv @ g

            while (E(F_temp, G, i, t, α) < e_i + sigma * c * g.T @ d):
                F_temp[i] = F_temp[i] + sigma * d
                sigma = p * sigma
            
            F[i] = F_temp + sigma * d
            q[i] = E(F, G, i, t, α)

            # Should we inverse by mesh instead of entire matrix?
            Q[i] = q[i]
            G[i] = np.linalg.inv(Q[i])
        
        # Out of loop, Save image of new mesh q
        
        # New mesh is now old mesh
        Q = np.copy(q)
        G = np.linalg.inv(Q)